# Installing python packages into Notebook

In [ ]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyyaml
!{sys.executable} -m pip install oyaml
!{sys.executable} -m pip install pandas

In [ ]:
import api

In [ ]:
import imp
imp.reload(api)

# Speciific App

In [ ]:
app = api.App("prod", "dmzvm")

In [ ]:
api.LinkTable(app.iterlinks(), api.f.endpoint("zone", "prod"))

# Runtime Review

In [ ]:
run = api.Runtime()

In [ ]:
api.Runtime.zones

## Load up data

In [ ]:
%%time
run = run.refresh()

## Summary Stats

In [ ]:
api.Table(run.stats())

### Alert Count

In [ ]:
sum([a["Num Links"] for a in run.stats()])

## All the links

In [ ]:
api.LinkTable(run.iterlinks())

In [ ]:
for zone in run.iterzones():
    for app in zone.iterapps():
        for link in app.iterlinks():
            print(link)

# Meta Policies and Suggestions

In [ ]:
api.mpr.run(run.iterlinks()).review()

## Pending Ones

In [ ]:
api.LinkTable(run.iterlinks(),
              api.f.nstate(None),
              #api.f.perimeter,
              api.f.endpoint("zone", "nightly-k8s"), api.f.endpoint("app", "^monitoring\."),
              api.f.neg(api.f.perimeter),
              #api.f.same_zone, #api.f.ltype(["AEG", "AIN"]),
              #api.f.neg(api.f.ltype(["AEG", "AIN"]), api.f.neg(api.f.same_zone)),
              #api.f.neg(api.f.server_non_ip),
              api.f.server_non_ip
             )

## Trying a new Meta Policy

In [ ]:
class MpTest:                                                                                                                                  
    policies = [                                                                
        api.AcceptLink(filters=[                                                    
                api.f.endpoint("app", "monitoring.alertmanager.alertmanager"),               
                api.f.ltype("NAE"),                                                 
                api.f.endpoint("process", "alertmanager", who="client"),              
                api.f.endpoint("dns_pattern", ":alertmanager-main-2.alertmanager-operated.monitoring.svc.cluster.local:", who="server"),
            ], changes=[                                                        
                ("server", "dns_pattern", ":.alertmanager-main-2.alertmanager-operated.monitoring.svc.cluster.local:"),         
            ]),                                                                                                                                
    ]    

In [ ]:
api.MetaPolicyRunner(MpTest).run(run.iterlinks()).review(MpTest)

# Relocation Example

In [ ]:
api.LinkTable(run.iterzones("nightly").iterapps("bendvm").relocate("prod", "bendvm").iterlinks())

In [ ]:
api.LinkTable(run.iterzones("nightly").iterapps("bendvm").to_lib().iterlinks())